### Goal: Download a sample OpenTopography LiDAR Tile to understand the data structure

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import os
import json
import hashlib
import requests
from pathlib import Path
import xml.etree.ElementTree as ET

from agents import Agent, Runner, WebSearchTool, CodeInterpreterTool, trace
from IPython.display import display, Markdown

In [4]:
task = """Download a sample OpenTopography LiDAR Tile, understand the data structure, and write a small report. 
Also write a Python code sample to list feasible request inputs, and a function that submits a request to download the file.

1. Download one OpenTopography LiDAR tile.
2. Run a single OpenAI o3/o4-mini or GPT-4.1 prompt on that data (e.g., “describe surface features
in plain English”).
3. Print model version and dataset ID.
"""

manager_prompt = f"""You are an elite researcher and programmer.
Your team's job is to efficiently complete the assigned task with precision and recall.
Your team can search the internet, read documentation, and do anything needed to get the task done."""

researcher_prompt = f"""You are an elite researcher and programmer. A Davinci-like Pythonista."""

In [14]:
research_agent = Agent(
    name="Researcher", 
    instructions=researcher_prompt, 
    tools=[
        WebSearchTool(), 
        CodeInterpreterTool(
            tool_config={"type": "code_interpreter", "container": {"type": "auto"}},
        )
    ]
)
manager_agent = Agent(
    name="Manager", 
    instructions=manager_prompt, 
    handoffs=[research_agent]
)
result = await Runner.run(manager_agent, task) 

In [ ]:
display(Markdown(result.final_output))

In [18]:
def query_opentopo_catalog():
    url = "https://portal.opentopography.org/API/otCatalog"
    params = {
        "productFormat": "PointCloud",
        "minx": -123.2,
        "miny": 44.8,
        "maxx": -121.9,
        "maxy": 45.7,
        "detail": "false",
        "outputFormat": "xml",
        "include_federated": "false"
    }
    headers = {
        "accept": "*/*"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response.text
    else:
        print(f"❌ Failed with status code {response.status_code}")
        print(response.text)

def parse_opentopo_xml(xml_string):
    root = ET.fromstring(xml_string)
    
    datasets = []
    for dataset in root.findall("Dataset"):
        name = dataset.findtext("name")
        opentopo_id = dataset.find("identifier/value").text
        alternate_name = dataset.findtext("alternateName")
        date_created = dataset.findtext("dateCreated")
        url = dataset.findtext("url")
        
        datasets.append({
            "name": name,
            "opentopo_id": opentopo_id,
            "alternate_name": alternate_name,
            "date_created": date_created,
            "url": url
        })
    
    return datasets

def query_globaldem(
  demtype="SRTMGL3",
  south=50,
  north=50.1,
  west=14.35,
  east=14.6,
  output_format="GTiff"
):
    url = "https://portal.opentopography.org/API/globaldem"
    params = {
        "demtype": demtype,
        "south": south,
        "north": north,
        "west": west,
        "east": east,
        "outputFormat": output_format,
        "API_Key": os.environ['OPENTOPOGRAPHY_API_KEY']
    }
    headers = {
        "accept": "*/*"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response
    else:
        print(f"❌ Failed with status code {response.status_code}")
        return response

def stable_query_hash(args):
    return hashlib.sha256(json.dumps(args, sort_keys=True).encode()).hexdigest()

In [6]:
res = query_opentopo_catalog()

In [7]:
parsed_datasets = parse_opentopo_xml(res)

for ds in parsed_datasets:
    print(f"{ds['name']} ({ds['opentopo_id']}) - {ds['alternate_name']} - Created: {ds['date_created']}")

Eagle Creek Fire Area in the Columbia River Gorge, Oregon (OTLAS.112018.26910.1) - OR18_Roering - Created: 2018-11-09
Oregon Department of Geology and Mineral Industries Lidar Program Data (OTLAS.022011.2994.1) - DOGAMI - Created: 2011-02-01


In [8]:
query_args = dict(
    demtype="SRTMGL3",
    south=50,
    north=50.1,
    west=14.35,
    east=14.6,
    output_format="GTiff"
)

res = query_globaldem(**query_args)

In [19]:
opentopo_data_root = Path.cwd() / "data" / "opentopography"
os.makedirs(opentopo_data_root, exist_ok=True)

manifest_file_path = opentopo_data_root / "manifest.json"

global_dem_data_root = opentopo_data_root / "globaldem"
os.makedirs(global_dem_data_root, exist_ok=True)

query_hash = stable_query_hash(query_args)

tif_file_path = f"{global_dem_data_root}/{query_hash}.tif"
with open(tif_file_path, "wb") as f:
    f.write(res.content)

if os.path.exists(manifest_file_path):
    with open(manifest_file_path, 'r') as f:
        manifest = json.load(f)
else:
    manifest = {}
manifest[query_hash] = {
    'local_file': tif_file_path,
    'remote_file': None,
    'args': query_args
}
with open(manifest_file_path, 'w') as f:
    json.dump(manifest, f)